# Tethys Engineering Development

The purpose of this development notebook is to capture the relevant workflow that culminated in the table transformations generated from the Tethys API call, which are deployed and continually developed in the software package. This notebook will be out of date if the API changes, or significant changes are made to the codebase that deviate from the table decomposition outlined and arrived at by this notebook. It may be helpful, however, as a detailed look at what is possible to draw from the API by doing a git checkout to understand the development cycle of the table composition arrived at here. This may also be a helpful starting palce for additional feature development using the API, and a git checkout to this commit may give a good starting place for additional branches that take development into these features using this development cycle as a starting place.

### Setup

In [1]:
import requests
import pandas as pd
import re

tethys_e_api = 'https://tethys-engineering.pnnl.gov/api/primre_export'
tethys_e_response = requests.get(tethys_e_api)
tethys_e_response_json = tethys_e_response.json()
tethys_e_dataframe = pd.DataFrame(tethys_e_response_json)

### Dev

In [2]:
tethys_e_response_json[0]

{'URI': 'https://tethys-engineering.pnnl.gov/node/4',
 'type': ['Document', 'Document/Journal Article'],
 'landingPage': 'https://tethys-engineering.pnnl.gov/publications/analytical-linear-modelization-buckled-undulating-membrane-tidal-energy-converter',
 'sourceURL': 'https://www.sciencedirect.com/science/article/pii/S0960148118306943',
 'title': 'Analytical linear modelization of a buckled undulating membrane tidal energy converter',
 'description': "This paper presents an analytical linear model developed to study the behaviour of a buckled membrane tidal energy converter. The Euler beam theory and the elongated body theory are used for the fluid structure interaction formulation. The effect of electromechanical converters used to convert the undulating motion into electrical energy is reproduced by adding a term equivalent to viscous material damping.  The influence of compression force, flaps and hanging conditions is studied, as well as the effects of simulated power take-off thr

In [3]:
tethys_e_dataframe[0:10]

,URI,type,landingPage,sourceURL,title,description,author,organization,originationDate,spatial,technologyType,tags,modifiedDate,signatureProject,attachment
0,https://tethys-engineering.pnnl.gov/node/4,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.sciencedirect.com/science/article/...,Analytical linear modelization of a buckled un...,This paper presents an analytical linear model...,"[Träsch, M., Déporte, A., Delacroix, S., Germa...",[],2019-01-01,[],"[Current, Current/Tidal]","[Engineering, Performance, Modeling]",2022-07-26 02:02:47,[],[]
1,https://tethys-engineering.pnnl.gov/node/6,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.sciencedirect.com/science/article/...,Estimating the optimum size of a tidal array a...,This paper investigates the optimum tidal ener...,"[Gonzalez-Gorbena, E., Pacheco, A., Plomaritis...","[University of Algarve, University of Cadiz]",2018-12-15,"{'coordinates': ['36.975374000000', '-7.876518...","[Current, Current/Tidal]","[Engineering, Array Effects, Performance, Depl...",2022-04-04 01:23:24,[],[]
2,https://tethys-engineering.pnnl.gov/node/7,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.sciencedirect.com/science/article/...,The potential of cross-stream aligned sub-arra...,A theoretical model is proposed for a row of s...,"[Cooke, S., Willden, R., Byrne, B.]",[University of Oxford],2016-06-03,[],"[Current, Current/Tidal]","[Engineering, Array Effects, Performance, Mode...",2022-04-04 01:23:24,[],[]
3,https://tethys-engineering.pnnl.gov/node/8,"[Document, Document/Workshop Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://ora.ox.ac.uk/objects/uuid:07f197f5-6b7...,Numerical modelling of two-scale turbine flow ...,Two-scale actuator disc theory is shown to pre...,"[Bonar, P., Venugopal, V., Borthwick, A., Adco...",[University of Oxford],2016-03-22,[],"[Current, Current/Tidal]","[Engineering, Performance, Modeling]",2022-04-04 01:23:24,[],[https://tethys-engineering.pnnl.gov/sites/def...
4,https://tethys-engineering.pnnl.gov/node/9,"[Document, Document/Conference Paper]",https://tethys-engineering.pnnl.gov/publicatio...,https://ieeexplore.ieee.org/document/5560384,An Optimum Operation and Mathematical Model of...,The hydro power is a renewable energy that had...,"[Fahmy, F.]",[Egypt National Research Center],2010-09-02,"{'coordinates': ['25.384808000000', '35.986809...","[Current, Current/Tidal]","[Engineering, Performance, Site Characterizati...",2023-08-04 06:30:13,[],[]
5,https://tethys-engineering.pnnl.gov/node/10,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.sciencedirect.com/science/article/...,Design and optimization for strength and integ...,Tidal turbine rotor blade fractures and failur...,"[Pengfei, L., Brian, V.]","[National Research Council Canada, Memorial Un...",2012-10-01,[],"[Current, Current/Axial Flow Turbine, Current/...","[Engineering, Structural, Modeling]",2022-04-04 01:23:24,[],[]
6,https://tethys-engineering.pnnl.gov/node/11,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.cambridge.org/core/journals/journa...,Performance of an ideal turbine in an inviscid...,Although wind and tidal turbines operate in tu...,"[Draper, S., Nishino, T. , Adcock, T., Taylor,...","[University of Western Australia, Cranfield Un...",2016-05-10,[],"[Current, Current/Axial Flow Turbine, Current/...","[Engineering, Control, Performance, Modeling]",2022-04-04 01:23:24,[],[]
7,https://tethys-engineering.pnnl.gov/node/12,"[Document, Document/Journal Article]",https://tethys-engineering.pnnl.gov/publicatio...,https://www.icevirtuallibrary.com/doi/10.1680/...,The power available to tidal turbines in an op...,Linear momentum actuator disc theory is extend...,"[Houlsby, G., Vogel, C.]",[University of Oxford],2017-01-11,[],"[Cur

### Clean

In [4]:
def find_entry_id(entry_uri):
    '''
    This function takes in the url of a MHKDR entry, and returns the entry_id of that page. 
    The 'entry_id' is the integer at the end of the url, which is unique to each MHKDR entry.
    The regex used in this function relies on the fact that the only number in the url is the id.
    '''
    rule = re.compile(r'\d+')
    matches_rule = rule.findall(entry_uri)
    entry_id = int(matches_rule[0])

    return entry_id

In [5]:
def construct_authors_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "author," connected to an "entry_id" that 
    may be called as a primary key to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    authors_of_entries = list(mhkdr_dataframe['author'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    authors = list()    # This list will contain duplicate authors when an author contributes to multiple entries
    
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])
        
        # Construct "author" column
        num_authors = len(authors_of_entries[i])
        for j in range(0, num_authors):
            entry_ids.append(entry_id)
            authors.append(authors_of_entries[i][j])
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'author':authors})
    
    return final_df

In [6]:
def construct_organizations_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "organization," connected to an "entry_id" that 
    may be called as a primary k4ey to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    orgs_of_entries = list(mhkdr_dataframe['organization'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    orgs = list()    # This list will contain duplicate authors when an author contributes to multiple entries
       
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])

        # Construct "author" column
        num_orgs = len(orgs_of_entries[i])
        for j in range(0, num_orgs):
            entry_ids.append(entry_id)
            orgs.append(orgs_of_entries[i][j])
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'organization':orgs})
    
    return final_df

In [7]:
def construct_tags_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "tags," connected to an "entry_id" that 
    may be called as a primary key to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    tags_of_entries = list(mhkdr_dataframe['tags'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    tags = list()    # This list will contain duplicate authors when an author contributes to multiple entries
    
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])
        
        # Construct "tag" column
        num_tags = len(tags_of_entries[i])
        for j in range(0, num_tags):
            entry_ids.append(entry_id)
            tags.append(tags_of_entries[i][j])
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'tag':tags})
    
    return final_df

### Cleaned - Tests

In [8]:
construct_authors_table(tethys_e_dataframe)

,entry_id,author
0,4,"Träsch, M."
1,4,"Déporte, A."
2,4,"Delacroix, S."
3,4,"Germain, G."
4,4,"Drevet, J."
...,...,...
31893,22430,"Mattiazzo, G."
31894,17432,PRIMRE
31895,17438,France Energies Marines
31896,17526,France Energies Marines


In [9]:
construct_organizations_table(tethys_e_dataframe)

,entry_id,organization
0,6,University of Algarve
1,6,University of Cadiz
2,7,University of Oxford
3,8,University of Oxford
4,9,Egypt National Research Center
...,...,...
13652,22428,Offshore Renewable Energy (ORE) Catapult
13653,22428,Crown Estate Scotland
13654,22429,Illinois Institute of Technology
13655,22429,Lancaster University


In [10]:
construct_tags_table(tethys_e_dataframe)

,entry_id,tag
0,4,Engineering
1,4,Performance
2,4,Modeling
3,6,Engineering
4,6,Array Effects
...,...,...
30671,17438,Engineering
30672,17438,Mooring
30673,17526,Engineering
30674,17526,Substructure
